In [1]:
# _September To lexie&amanda 2018: 9/1/18-10/11/18

import pandas as pd
import numpy as np
import datetime
import pandas as pd
from googleads import adwords
import google_analytics_reporting_api as ga
import os

week="_September_for_TV" 

Start_Date_overall='2018-09-01'
End_Data_overall='2018-10-11'

Start_Date_1='2018-09-01'
End_Date_1='2018-09-20'

Start_Date_2='2018-09-21'
End_Date_2='2018-10-11'


today_str=str(datetime.datetime.now().date())
writer_folder="/home/jian/Projects/Saatva/To_Lexie_and_Connor/output/running_on_"+today_str+"/"
try:
    os.stat(writer_folder)
except:
    os.mkdir(writer_folder)

writer_folder="/home/jian/Projects/Saatva/To_Lexie_and_Connor/output/running_on_"+today_str+"/"+week[1:]+"/"
try:
    os.stat(writer_folder)
except:
    os.mkdir(writer_folder)

# Google Analytics

In [2]:
analytics = ga.initialize_analyticsreporting('/home/jian/Projects/Saatva/GA/GA_api_jubapluscc@gmail.com.json')

def num_func_trans(df):
    if 'ga:sessions' in df.columns:
        df['ga:sessions']=df['ga:sessions'].apply(lambda x: int(x.replace(",","")))
        df['ga:sessions']=df['ga:sessions'].astype(int)

    if 'ga:adClicks' in df.columns:
        df['ga:adClicks']=df['ga:adClicks'].apply(lambda x: int(x.replace(",","")))
        df['ga:adClicks']=df['ga:adClicks'].astype(int)

    if 'ga:impressions' in df.columns:
        df['ga:impressions']=df['ga:impressions'].apply(lambda x: int(x.replace(",","")))
        df['ga:impressions']=df['ga:impressions'].astype(int)
    
    if 'ga:transactions' in df.columns:
        df['ga:transactions']=df['ga:transactions'].apply(lambda x: int(x.replace(",","")))
        df['ga:transactions']=df['ga:transactions'].astype(int)
    
    if 'ga:transactionRevenue' in df.columns:
        df['ga:transactionRevenue']=df['ga:transactionRevenue'].apply(lambda x: float(x.replace(",","")))
        df['ga:transactionRevenue']=df['ga:transactionRevenue'].astype(float)

    
    return df


In [3]:
# Saatva.com
# dateHour, sourceMedium      sessions, adClicks, impressions
ga_data_1=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='122377905',start_date=Start_Date_1, end_date=End_Date_1,
                                            ga_metrics=['ga:sessions','ga:adClicks','ga:impressions','ga:transactions','ga:transactionRevenue'],
                                            ga_dimensions=['ga:dateHour','ga:sourceMedium'])

ga_data_2=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='122377905',start_date=Start_Date_2, end_date=End_Date_2,
                                            ga_metrics=['ga:sessions','ga:adClicks','ga:impressions','ga:transactions','ga:transactionRevenue'],
                                            ga_dimensions=['ga:dateHour','ga:sourceMedium'])


ga_data_saatva=ga_data_1.append(ga_data_2)
ga_data_saatva['View']="Saatva.com"
ga_data_saatva['View_ID']="122377905"
ga_data_saatva.shape

(7972, 9)

In [4]:
# Saatvamattress.com
# dateHour, sourceMedium      sessions, adClicks, impressions
ga_data_1=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='33605697',start_date=Start_Date_1, end_date=End_Date_1,
                                            ga_metrics=['ga:sessions','ga:adClicks','ga:impressions','ga:transactions','ga:transactionRevenue','ga:transactions','ga:transactionRevenue'],
                                            ga_dimensions=['ga:dateHour','ga:sourceMedium'])

ga_data_2=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='33605697',start_date=Start_Date_2, end_date=End_Date_2,
                                            ga_metrics=['ga:sessions','ga:adClicks','ga:impressions','ga:transactions','ga:transactionRevenue','ga:transactions','ga:transactionRevenue'],
                                            ga_dimensions=['ga:dateHour','ga:sourceMedium'])


ga_data_SM=ga_data_1.append(ga_data_2)
ga_data_SM['View']="Saatvamattress.com"
ga_data_SM['View_ID']="33605697"
ga_data_SM.shape

(54631, 9)

In [5]:
ga_data=ga_data_SM.append(ga_data_saatva)
ga_data=num_func_trans(ga_data)

In [6]:
ga_data['Source']=ga_data['ga:sourceMedium'].apply(lambda x: x.split(" / ")[0])
ga_data['Medium']=ga_data['ga:sourceMedium'].apply(lambda x: x.split(" / ")[1])

In [7]:
# msn in 'social'
# mail.yahoo.com / referral in 'referral'
# Social: 'msn','instagram','pinterest','facebook
                                                
ga_data['Defined_Media']=np.nan
ga_data['Defined_Media']=np.where(((ga_data['ga:sourceMedium'].str.contains('youtube.')) | (ga_data['Medium']=="social")),"Social",
                                  np.where(((ga_data['Source'].isin(['msn','instagram','Instagram','pinterest','facebook'])) & (ga_data['Medium']=='cpc')),"Social",
                                           np.where(ga_data['ga:sourceMedium'].str.contains('criteo'),"Retarget",
                                                    np.where(ga_data['Source']=="(direct)","Direct",
                                                             np.where(((ga_data['Source'].isin(['google','bing','yahoo'])) & (ga_data['Medium']=='cpc')),"SEM",
                                                                      np.where(((ga_data['Source'].isin(['google','bing','yahoo'])) & (ga_data['Medium']=='organic')),"SEO",
                                                                               np.where(ga_data['Source']=="powerInbox","Email",
                                                                                        np.where((ga_data['ga:sourceMedium'].str.contains('email')) & (~ga_data['ga:sourceMedium'].isin(["taboola / townnews-wvgazettemail","query.hemailaccessonline.com / referral"]) | (ga_data['ga:sourceMedium'].str.contains("mail.") )),"Email",
                                                                                                 np.where(ga_data['ga:sourceMedium'].str.contains('taboola'),"Video",
                                                                                                          np.where(ga_data['Medium']=="referral","Referral",
                                                                                                                   np.where(ga_data['Medium'].isin(["partner","partners"]),"partner",
                                                                                                                            np.where(ga_data['ga:sourceMedium'].str.contains('display'),"Display","Others")
                                                                                                                            )
                                                                                                                   )
                                                                                                          )
                                                                                                )
                                                                                       )
                                                                              )
                                                                     )
                                                            )
                                                   )
                                          )
                                 )
                                                         


In [8]:
GA_taxonomy=ga_data[['ga:sourceMedium','Source','Medium','Defined_Media']].drop_duplicates()
GA_taxonomy.reset_index(inplace=True)
GA_taxonomy=GA_taxonomy.sort_values('Defined_Media')
del GA_taxonomy['index']

In [9]:
ga_data.head(2)

,ga:adClicks,ga:dateHour,ga:impressions,ga:sessions,ga:sourceMedium,ga:transactionRevenue,ga:transactions,View,View_ID,Source,Medium,Defined_Media
0,0,2018090100,0,65,(direct) / (none),1717.0,1,Saatvamattress.com,33605697,(direct),(none),Direct
1,0,2018090100,0,24,(not set) / partner,2454.0,1,Saatvamattress.com,33605697,(not set),partner,partner


In [10]:
ga_agg_session=ga_data.groupby(['ga:dateHour','Defined_Media'])['ga:sessions'].sum().to_frame()
ga_agg_session.reset_index(inplace=True)
ga_agg_wide_session=ga_agg_session.pivot('ga:dateHour','Defined_Media','ga:sessions')
ga_agg_wide_session.reset_index(inplace=True)
ga_agg_wide_session['time_hour']=ga_agg_wide_session['ga:dateHour'].apply(lambda x:datetime.datetime.strptime(x,"%Y%m%d%H"))

In [11]:
ga_agg_transaction=ga_data.groupby(['ga:dateHour','Defined_Media'])['ga:transactions'].sum().to_frame()
ga_agg_transaction.reset_index(inplace=True)
ga_agg_wide_trans=ga_agg_transaction.pivot('ga:dateHour','Defined_Media','ga:transactions')
ga_agg_wide_trans.reset_index(inplace=True)
ga_agg_wide_trans['time_hour']=ga_agg_wide_trans['ga:dateHour'].apply(lambda x:datetime.datetime.strptime(x,"%Y%m%d%H"))

In [12]:
ga_agg_Revenue=ga_data.groupby(['ga:dateHour','Defined_Media'])['ga:transactionRevenue'].sum().to_frame()
ga_agg_Revenue.reset_index(inplace=True)
ga_agg_wide_Revenue=ga_agg_Revenue.pivot('ga:dateHour','Defined_Media','ga:transactionRevenue')
ga_agg_wide_Revenue.reset_index(inplace=True)
ga_agg_wide_Revenue['time_hour']=ga_agg_wide_Revenue['ga:dateHour'].apply(lambda x:datetime.datetime.strptime(x,"%Y%m%d%H"))


In [13]:
writer=pd.ExcelWriter(writer_folder+'/Saatva_S_and_SM_GA_by_Media_'+Start_Date_1+'_to_'+End_Date_2+week+'.xlsx', engine='xlsxwriter')

In [14]:
# Combined both Saatva.com and Saatvamattress.com
ga_agg_wide_session.to_excel(writer,"Session_hour_media",index=False)
ga_agg_wide_trans.to_excel(writer,"Trans_hour_media",index=False)
ga_agg_wide_Revenue.to_excel(writer,"Revenue_hour_media",index=False)

ga_data.to_excel(writer,"data",index=False)
GA_taxonomy.to_excel(writer,"taxonomy",index=False)
writer.save()
GA_taxonomy.to_csv(writer_folder+'/Saatva_S_and_SM_GA_taxonomy_'+Start_Date_1+'_to_'+End_Date_2+week+'.csv',index=False)

/usr/local/lib/python3.6/site-packages/xlsxwriter/worksheet.py:831: UserWarning: Ignoring URL 'https://googleads.g.doubleclick.net/pagead/ads?client=ca-pub-0827640844035835&output=html&h=90&slotname=5853479398&adk=142861634&adf=3873593895&w=728&lmt=1535776720&guci=2.2.0.0.2.2.0&url=http://malayalam.changathi.com/&flash=0&wgl=1&dt=1535776720183&bpp=6&bdt=397&fdt=85&idt=88&shv=r20180827&cbv=r20180604&saldr=sa&abxe=1&prev_slotnames=3127491512&correlator=6457407368863&frm=20&pv=1&ga_vid=485441565.1535776720&ga_sid=1535776720&ga_hid=1658319657&ga_fc=0&iag=0&icsg=691240&dssz=14&mdo=0&mso=0&u_tz=-420&u_his=1&u_java=0&u_h=568&u_w=320&u_ah=568&u_aw=320&u_cd=32&u_nplug=0&u_nmime=0&adx=54&ady=606&biw=980&bih=1507&scr_x=0&scr_y=0&eid=21060853,21061011&oid=3&rx=0&eae=0&fc=528&brdim=0,0,0,0,320,0,0,0,980,1507&vis=1&rsz=||eE|&abl=CS&ppjl=t&pfx=0&fu=16&bc=7&ifi=2&fsb=1&xpc=8wxieBIvaQ&p=http://malayalam.changathi.com&dtd=98%20/%20partner' with link or location/anchor > 255 characters since it exceeds E

/usr/local/lib/python3.6/site-packages/xlsxwriter/worksheet.py:831: UserWarning: Ignoring URL 'https://www.buzzfeed.com/colingorenstein/products-under-20-upgrade-space?origin=niftynl&utm_medium=email&utm_campaign=Nifty%20Home%209418&utm_content=Nifty%20Home%209418+CID_429c3524894a4e0700712f7cd3a8b45b&utm_source=BuzzFeed%20Newsletters&utm_term=.kqE0J8VBK%20/%20partner' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
/usr/local/lib/python3.6/site-packages/xlsxwriter/worksheet.py:831: UserWarning: Ignoring URL 'https://www.amazon.com/Pulse-1013-GL-Shower-Silver-Hardware/dp/B00IUCHF8K/ref=as_at?creativeASIN=B00IUCHF8K&imprToken=0.hPqjjeZJXXNaDbDORosg&slotNum=14&s=hi&ie=UTF8&qid=1513344701&sr=1-1&keywords=B00IUCHF8K&linkCode=w61&tag=haciko-20&linkId=1e93f73d2239a7b0af43b9c36db1b891%20/%20partner' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
/usr/local/lib/python3.6/site

/usr/local/lib/python3.6/site-packages/xlsxwriter/worksheet.py:831: UserWarning: Ignoring URL 'https://www.thesleepjudge.com/editors-choice/?pt_source=googleleads&pt_medium=cpc&pt_campaign=%5bADL%5d%20%5bNon-Brand%5d%20%5bUS%5d%20Mattresses%20(Exact)&pt_adgroup=best%20mattresses%202018&pt_keyword=best%20mattresses%202018&gclid=Cj0KCQjwidPcBRCGARIsALM--ePO9ur0h-5Amqvzo1pS89Bj4nbydcnUwOHAlO3P-0MTGKGBniPeFb0aAjtYEALw_wcB%20/%20partner' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
/usr/local/lib/python3.6/site-packages/xlsxwriter/worksheet.py:831: UserWarning: Ignoring URL 'https://www.thesleepjudge.com/editors-choice/?pt_source=googleleads&pt_medium=cpc&pt_campaign=%5bADL%5d%20%5bNon-Brand%5d%20%5bUS%5d%20Mattresses%20(Exact)&pt_adgroup=best%20mattresses%202018&pt_keyword=best%20mattresses%202018&gclid=EAIaIQobChMI8YSrhMmu3QIVxoCfCh13VAn9EAAYASAAEgK6w_D_BwE%20/%20partner' with link or location/anchor > 255 characters since it 

/usr/local/lib/python3.6/site-packages/xlsxwriter/worksheet.py:831: UserWarning: Ignoring URL 'https://googleads.g.doubleclick.net/pagead/ads?ad_block=2&client=ca-pub-6219811747049371&output=html&h=250&w=300&loeid=9471239,9475653,9486211,9488571,23722183,23726564,23726888,23733598,23734874,23735275,23743669,23745798,23749951,23751755,23754122,23754721,23755135,23756714,23757157,23757887,23759783,23760721,23763106,23763475,23763966,23764470,23765017&pucrd=CgYIABAAGAB4Ag&channel=0854550287+afc_on_page+afv_user_bgfilms+afv_user_id_JHA_jMfCvEnv-3kRjTCQXw+yt_cid_11344300+yt_mpvid_Jy3q9tElDrA-TvPM+yt_no_ap+yt_no_wa+ytdevice_1+ytdevicever_2.20180913+ytel_detailpage+ytps_default&ad_type=image&format=300x250_as&url=http://www.youtube.com/video/nKKznGaN7PI&flash=0&host=ca-yt-host-pub-2279761097731703&alternate_ad_url=https://www.youtube.com/ad_frame?action=auto-collapse-kevlar-companion&hl=en&video_doc_id=yt_nKKznGaN7PI&wgl=1&dbp=ChZ6dnpYZVhzb3I2eFYzYVhYcGR0RTh3EAEgASgA&lact=-1&yt_pt=APb3F2_1cQ8

/usr/local/lib/python3.6/site-packages/xlsxwriter/worksheet.py:831: UserWarning: Ignoring URL 'https://www.consumersadvocate.org/mattress/a/best-mattress?&pd=true&keyword=best%20mattress&gca_campaignid=1016369000&gca_adgroupid=51320554757&gca_matchtype=p&gca_network=s&gca_device=c&gca_adposition=1t1&gca_loc_interest_ms=&gca_loc_physical_ms=9012047&gclid=CjwKCAjwxILdBRBqEiwAHL2R871t_iFDBejzXA5pJWHWN6QECCk5YrW_CCz1h3gvwWK8W3l_q5PNNRoC4bcQAvD_BwE%20/%20partner' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
/usr/local/lib/python3.6/site-packages/xlsxwriter/worksheet.py:831: UserWarning: Ignoring URL 'https://googleads.g.doubleclick.net/pagead/ads?client=ca-pub-7904173988271272&output=html&h=280&slotname=6766256624&adk=4279750640&adf=3858009952&w=375&fwrn=4&fwrnh=100&lmt=1537247177&pucrd=CgwIABAAGAMgACgAOAESAhgHeAM&rafmt=3&guci=2.2.0.0.2.2.0&format=375x280&url=https://goldprice.org/&region=responsive_resize&flash=0&fwr=0&pra=3&rh

...

/usr/local/lib/python3.6/site-packages/xlsxwriter/worksheet.py:831: UserWarning: Ignoring URL 'https://www.thesleepjudge.com/editors-choice/?pt_source=googleleads&pt_medium=cpc&pt_campaign=%5bADL%5d%20%5bNon-Brand%5d%20%5bUS%5d%20Mattresses%20(Exact)&pt_adgroup=best%20mattresses%202018&pt_keyword=best%20mattresses%202018&gclid=Cj0KCQjwlqLdBRCKARIsAPxTGaXrZE6A8vV8qrJ-6Zb2UiYVs5Z7hv7vy0K-q6aRYr4JSr4VAFmUFrQaAoWXEALw_wcB' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
/usr/local/lib/python3.6/site-packages/xlsxwriter/worksheet.py:831: UserWarning: Ignoring URL 'https://www.thesleepjudge.com/editors-choice/?pt_source=googleleads&pt_medium=cpc&pt_campaign=%5bADL%5d%20%5bNon-Brand%5d%20%5bUS%5d%20Mattresses%20(Exact)&pt_adgroup=best%20mattresses%202018&pt_keyword=best%20mattresses%202018&gclid=EAIaIQobChMIkeyBzuXT3QIV1I2zCh0O3wJeEAAYAiAAEgLgSPD_BwE' with link or location/anchor > 255 characters since it exceeds Excel's limit for UR

/usr/local/lib/python3.6/site-packages/xlsxwriter/worksheet.py:831: UserWarning: Ignoring URL 'https://www.consumersadvocate.org/mattress/a/best-mattress?&pd=true&keyword=mattress%20reviews&gca_campaignid=1016369000&gca_adgroupid=50955604065&gca_matchtype=p&gca_network=g&gca_device=c&gca_adposition=1t3&gca_loc_interest_ms=&gca_loc_physical_ms=9004523&gclid=EAIaIQobChMIhoXaxOfh3QIVBiSGCh2yVwEeEAAYAyAAEgK1qfD_BwE' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
/usr/local/lib/python3.6/site-packages/xlsxwriter/worksheet.py:831: UserWarning: Ignoring URL 'https://www.consumersadvocate.org/mattress/a/best-mattress?&pd=true&keyword=best%20mattress&gca_campaignid=1016369000&gca_adgroupid=51320554757&gca_matchtype=p&gca_network=g&gca_device=c&gca_adposition=1t3&gca_loc_interest_ms=&gca_loc_physical_ms=9002254&gclid=Cj0KCQjwxbzdBRCoARIsACzIK2n6LpbUZQAO1IfV0Fb9yk0WcF3KepnR64UxfoDdFuvt1WtXCobsnIIaAn-lEALw_wcB' with link or location/anch

/usr/local/lib/python3.6/site-packages/xlsxwriter/worksheet.py:831: UserWarning: Ignoring URL 'https://www.thesleepjudge.com/editors-choice/?pt_source=googleleads&pt_medium=cpc&pt_campaign=%5bADL%5d%20%5bNon-Brand%5d%20%5bUS%5d%20Mattresses%20(Exact)&pt_adgroup=best%20mattresses%202018&pt_keyword=best%20mattresses%202018&gclid=Cj0KCQjw9NbdBRCwARIsAPLsnFZ1pLGAR0kpo1q2GfMNoaHlBZzCEhujBRkoIcAc8jl9iWBOiSoCF9MaAlGqEALw_wcB' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
/usr/local/lib/python3.6/site-packages/xlsxwriter/worksheet.py:831: UserWarning: Ignoring URL 'https://www.thesleepjudge.com/editors-choice/?pt_source=googleleads&pt_medium=cpc&pt_campaign=%5bADL%5d%20%5bNon-Brand%5d%20%5bUS%5d%20Mattresses%20(Exact)&pt_adgroup=best%20mattresses%202018&pt_keyword=best%20mattresses%202018&gclid=EAIaIQobChMItbKcr4Du3QIVBySGCh3ghwawEAAYASAAEgLdPPD_BwE' with link or location/anchor > 255 characters since it exceeds Excel's limit for UR

/usr/local/lib/python3.6/site-packages/xlsxwriter/worksheet.py:831: UserWarning: Ignoring URL 'https://www.consumersadvocate.org/mattress/a/best-mattress?&pd=true&keyword=best%20mattress%20reviews&gca_campaignid=1016369000&gca_adgroupid=51627636938&gca_matchtype=e&gca_network=s&gca_device=c&gca_adposition=1t2&gca_loc_interest_ms=&gca_loc_physical_ms=9031023&gclid=EAIaIQobChMIjdb39qn43QIVBTJpCh3QNgRNEAAYAiAAEgJCEPD_BwE' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
/usr/local/lib/python3.6/site-packages/xlsxwriter/worksheet.py:831: UserWarning: Ignoring URL 'https://www.consumersadvocate.org/mattress/a/best-mattress?&pd=true&keyword=best%20mattress&gca_campaignid=1016369000&gca_adgroupid=51320554757&gca_matchtype=p&gca_network=s&gca_device=c&gca_adposition=1t1&gca_loc_interest_ms=1026759&gca_loc_physical_ms=9027993&gclid=EAIaIQobChMI2-r9_qn43QIVgRlpCh0h4AFuEAAYASAAEgL6RPD_BwE' with link or location/anchor > 255 characters sin

/usr/local/lib/python3.6/site-packages/xlsxwriter/worksheet.py:831: UserWarning: Ignoring URL 'http://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=8&ved=0CCEQfjcnVkSDdXdGRvV1k4R0Vv&url=http://click.communications.onekingslane.com/?qs=1ef0eff5a76d9ac3885268803a71adff349ae0896a140c25e845377bee47798bf34bcda8cba839f6ce05d18f90c9f8889c094ee0cea792b5261da9c596042b70&ei=TWpQWlQydjZuRFcx&usg=AFQjaEhJOE5yN0VpRVly' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))


In [15]:
GA_taxonomy.to_csv(writer_folder+'/Saatva_S_and_SM_GA_by_Media_Taxonomy_'+Start_Date_1+'_to_'+End_Date_2+week+'.csv')

In [16]:
media_col=ga_agg_wide_session.columns.tolist()[1:(len(ga_agg_wide_session.columns)-1)]

In [17]:
def write_csv_output(df,Matrics):
    ga_output_csv=df.copy()
    ga_output_csv['Date']=ga_output_csv['ga:dateHour'].apply(lambda x:int(x[0:8]))
    ga_output_csv['Hour']=ga_output_csv['ga:dateHour'].apply(lambda x:int(x[8:10]))
    del ga_output_csv['ga:dateHour']
    ga_output_csv=ga_output_csv.sort_values(['Date','Hour'],ascending=True)
    ga_output_csv=ga_output_csv[['Date','Hour']+media_col]
    ga_output_csv=ga_output_csv.fillna(0)
    ga_output_csv.to_csv(writer_folder+'/Saatva_S_and_SM_GA_by_Media_'+Start_Date_1+'_to_'+End_Date_2+week+"_"+Matrics+'.csv',index=False)

In [18]:
write_csv_output(ga_agg_wide_session,"Session")
write_csv_output(ga_agg_wide_trans,"Trans")
write_csv_output(ga_agg_wide_Revenue,"Revenue")


In [19]:
ga_agg_session_by_View=ga_data.groupby(['View','View_ID','ga:dateHour','Defined_Media'])['ga:sessions'].sum().unstack('Defined_Media')
ga_agg_session_by_View.reset_index(inplace=True)
ga_agg_session_by_View=ga_agg_session_by_View.fillna(0)
ga_agg_session_by_View.to_csv(writer_folder+'/Saatva_S_and_SM_GA_by_Media_by_View'+Start_Date_1+'_to_'+End_Date_2+week+"_Sessions"+'.csv',index=False)




# AdWords keywords from GA

In [20]:
from fuzzywuzzy import fuzz
# fuzz 60 ratio score thredhold
writer=pd.ExcelWriter(writer_folder+"KeyWords_From_GA+"+Start_Date_1+'_to_'+End_Date_2+week+".xlsx")

/usr/local/lib/python3.6/site-packages/fuzzywuzzy/fuzz.py:35: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [21]:
# Saatva.com
# ga:date ga:keyword ga:impressions ga:adClicks
ga_data_Ads_Saatva_1=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='122377905',start_date=Start_Date_1, end_date=End_Date_1,
                                            ga_metrics=['ga:adClicks','ga:impressions'],
                                            ga_dimensions=['ga:date','ga:keyword'])
ga_data_Ads_Saatva_2=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='122377905',start_date=Start_Date_2, end_date=End_Date_2,
                                            ga_metrics=['ga:adClicks','ga:impressions'],
                                            ga_dimensions=['ga:date','ga:keyword'])
ga_data_Ads_Saatva=ga_data_Ads_Saatva_1.append(ga_data_Ads_Saatva_2)
ga_data_Ads_Saatva['key_word_list']=ga_data_Ads_Saatva['ga:keyword'].apply(lambda x: x.split(" "))
ga_data_Ads_Saatva['Brand_Ratio']=ga_data_Ads_Saatva['key_word_list'].apply(lambda x: max([fuzz.ratio("saatva",z) for z in x]))
ga_data_Ads_Saatva['key_words_group']=np.where(ga_data_Ads_Saatva['Brand_Ratio']>60,"Brand","Non_Brand")

taxonomy_Saatva=ga_data_Ads_Saatva[['ga:keyword','key_words_group']].drop_duplicates()
taxonomy_Saatva['website']="Saatva.com"
ga_data_Ads_Saatva['ga:adClicks']=ga_data_Ads_Saatva['ga:adClicks'].astype(int)
ga_data_Ads_Saatva['ga:impressions']=ga_data_Ads_Saatva['ga:impressions'].astype(int)
output_Saatva=ga_data_Ads_Saatva.groupby(["ga:date","ga:keyword","key_words_group"])['ga:adClicks','ga:impressions'].sum().reset_index()
output_Saatva_Brand=output_Saatva[output_Saatva['key_words_group']=="Brand"].sort_values('ga:date').reset_index()
del output_Saatva_Brand['index']

output_Saatva_Brand.to_excel(writer,"Saatva_Brand",index=False)
output_Saatva.to_excel(writer,"Saatva_Keywords",index=False)

In [22]:
# SaatvaMattress.com
# ga:date ga:keyword ga:impressions ga:adClicks
ga_data_Ads_SaatvaMattress_1=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='33605697',start_date=Start_Date_1, end_date=End_Date_1,
                                            ga_metrics=['ga:adClicks','ga:impressions'],
                                            ga_dimensions=['ga:date','ga:keyword'])
ga_data_Ads_SaatvaMattress_2=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='33605697',start_date=Start_Date_2, end_date=End_Date_2,
                                            ga_metrics=['ga:adClicks','ga:impressions'],
                                            ga_dimensions=['ga:date','ga:keyword'])
ga_data_Ads_SaatvaMattress=ga_data_Ads_SaatvaMattress_1.append(ga_data_Ads_SaatvaMattress_2)
ga_data_Ads_SaatvaMattress['key_word_list']=ga_data_Ads_SaatvaMattress['ga:keyword'].apply(lambda x: x.split(" "))
ga_data_Ads_SaatvaMattress['Brand_Ratio']=ga_data_Ads_SaatvaMattress['key_word_list'].apply(lambda x: max([fuzz.ratio("saatva",z) for z in x]))
ga_data_Ads_SaatvaMattress['key_words_group']=np.where(ga_data_Ads_SaatvaMattress['Brand_Ratio']>60,"Brand","Non_Brand")

taxonomy_SaatvaMattress=ga_data_Ads_SaatvaMattress[['ga:keyword','key_words_group']].drop_duplicates()
taxonomy_SaatvaMattress['website']="SaatvaMattress.com"
ga_data_Ads_SaatvaMattress['ga:adClicks']=ga_data_Ads_SaatvaMattress['ga:adClicks'].astype(int)
ga_data_Ads_SaatvaMattress['ga:impressions']=ga_data_Ads_SaatvaMattress['ga:impressions'].astype(int)
output_SaatvaMattress=ga_data_Ads_SaatvaMattress.groupby(["ga:date","ga:keyword","key_words_group"])['ga:adClicks','ga:impressions'].sum().reset_index()
output_SaatvaMattress_Brand=output_SaatvaMattress[output_SaatvaMattress['key_words_group']=="Brand"].sort_values('ga:date').reset_index()
del output_SaatvaMattress_Brand['index']

output_SaatvaMattress_Brand.to_excel(writer,"SaatvaMattress_Brand",index=False)
output_SaatvaMattress.to_excel(writer,"SaatvaMattress_Keywords",index=False)

In [23]:
taxonomy=taxonomy_Saatva.append(taxonomy_SaatvaMattress)
taxonomy.to_excel(writer,"taxonomy",index=False)
writer.save()

# Google AdWords

In [24]:
def get_client(config_path='googleads_saatva.yaml'):
    return adwords.AdWordsClient.LoadFromStorage(config_path)

def download_report_as_stream(client, report_type, fields, start_date, end_date):
    '''Download Adwords report, then save as csv file to base_dir
    Args:
        client: adwords.AdWordsClient.LoadFromStorage()
        report_type: str eg. 'KEYWORDS_PERFORMANCE_REPORT'
        fields: list eg. ["CampaignId", "CampaignName", ... ]
        start_date: str, eg. '2018-02-11'
        end_date: str, eg.
    Return:
        str, path to csv file
    '''
    report_downloader = client.GetReportDownloader(version='v201802')
    # Create report definition.
    report = {
        'reportName': report_type,
        'dateRangeType': 'CUSTOM_DATE',
        'reportType': report_type,
        'downloadFormat': 'CSV',
        'selector': {
            'fields': fields,
            'dateRange': {
                'min': start_date,
                'max': end_date}
        }
    }

    # save output
    return report_downloader.DownloadReportAsStream(report, 
        skip_report_header=True, skip_column_header=False,
        skip_report_summary=True)



In [25]:
client_Saatva = get_client(config_path='/home/jian/Projects/Saatva/To_Lexie_and_Connor/googleads_saatva_JL_Saatva.yaml')
report_type = 'CAMPAIGN_PERFORMANCE_REPORT'
fields = ["AccountDescriptiveName","CampaignName","Date","HourOfDay","AdvertisingChannelType","AdNetworkType1","AdNetworkType2","Impressions", "Clicks", "Cost"]
response_Saatva = download_report_as_stream(client_Saatva, report_type, fields, Start_Date_overall, End_Data_overall)
Saatva_Click = pd.read_csv(response_Saatva)
Saatva_Click['Cost']=Saatva_Click['Cost'].apply(lambda x: x/1000000)
Saatva_Click['Channel_Network']=Saatva_Click['Advertising Channel']+"|"+Saatva_Click['Network']+"|"+Saatva_Click['Network (with search partners)']

In [26]:
client_Saatva_Branding = get_client(config_path='/home/jian/Projects/Saatva/To_Lexie_and_Connor/googleads_saatva_JL_Saatva_Branding.yaml')
report_type = 'CAMPAIGN_PERFORMANCE_REPORT'
fields = ["AccountDescriptiveName","CampaignName","Date","HourOfDay","AdvertisingChannelType","AdNetworkType1","AdNetworkType2","Impressions", "Clicks", "Cost"]
response_Saatva_Branding = download_report_as_stream(client_Saatva_Branding, report_type, fields, Start_Date_overall, End_Data_overall)
Saatva_Brand_Click = pd.read_csv(response_Saatva_Branding)
Saatva_Brand_Click['Cost']=Saatva_Brand_Click['Cost'].apply(lambda x: x/1000000)
Saatva_Brand_Click['Channel_Network']=Saatva_Brand_Click['Advertising Channel']+"|"+Saatva_Brand_Click['Network']+"|"+Saatva_Brand_Click['Network (with search partners)']


In [27]:
client_Saatva_Competitors = get_client(config_path='/home/jian/Projects/Saatva/To_Lexie_and_Connor/googleads_saatva_JL_Saatva_Competitors.yaml')
report_type = 'CAMPAIGN_PERFORMANCE_REPORT'
fields = ["AccountDescriptiveName","CampaignName","Date","HourOfDay","AdvertisingChannelType","AdNetworkType1","AdNetworkType2","Impressions", "Clicks", "Cost"]
response_Saatva_Competitor = download_report_as_stream(client_Saatva_Competitors, report_type, fields, Start_Date_overall, End_Data_overall)
Saatva_Competitor_Click = pd.read_csv(response_Saatva_Competitor)
Saatva_Competitor_Click['Cost']=Saatva_Competitor_Click['Cost'].apply(lambda x: x/1000000)
Saatva_Competitor_Click['Channel_Network']=Saatva_Competitor_Click['Advertising Channel']+"|"+Saatva_Competitor_Click['Network']+"|"+Saatva_Competitor_Click['Network (with search partners)']



In [28]:
client_TheSaatvaCompany = get_client(config_path='/home/jian/Projects/Saatva/To_Lexie_and_Connor/googleads_saatva_JL_TheSaatvaCompany.yaml')
report_type = 'CAMPAIGN_PERFORMANCE_REPORT'
fields = ["AccountDescriptiveName","CampaignName","Date","HourOfDay","AdvertisingChannelType","AdNetworkType1","AdNetworkType2","Impressions", "Clicks", "Cost"]
response_TheSaatvaCompany = download_report_as_stream(client_TheSaatvaCompany, report_type, fields, Start_Date_overall, End_Data_overall)
TheSaatvaCompany_Click = pd.read_csv(response_TheSaatvaCompany)
TheSaatvaCompany_Click['Cost']=TheSaatvaCompany_Click['Cost'].apply(lambda x: x/1000000)
TheSaatvaCompany_Click['Channel_Network']=TheSaatvaCompany_Click['Advertising Channel']+"|"+TheSaatvaCompany_Click['Network']+"|"+TheSaatvaCompany_Click['Network (with search partners)']



In [29]:
client_TheSaatvaCompanyBranding = get_client(config_path='/home/jian/Projects/Saatva/To_Lexie_and_Connor/googleads_saatva_JL_TheSaatvaCompanyBranding.yaml')
report_type = 'CAMPAIGN_PERFORMANCE_REPORT'
fields = ["AccountDescriptiveName","CampaignName","Date","HourOfDay","AdvertisingChannelType","AdNetworkType1","AdNetworkType2","Impressions", "Clicks", "Cost"]
response_TheSaatvaCompanyBranding = download_report_as_stream(client_TheSaatvaCompanyBranding, report_type, fields, Start_Date_overall, End_Data_overall)
TheSaatvaCompanyBranding_Click = pd.read_csv(response_TheSaatvaCompanyBranding)
TheSaatvaCompanyBranding_Click['Cost']=TheSaatvaCompanyBranding_Click['Cost'].apply(lambda x: x/1000000)
TheSaatvaCompanyBranding_Click['Channel_Network']=TheSaatvaCompanyBranding_Click['Advertising Channel']+"|"+TheSaatvaCompanyBranding_Click['Network']+"|"+TheSaatvaCompanyBranding_Click['Network (with search partners)']



In [30]:
Saatva_Competitor_Click=Saatva_Competitor_Click[Saatva_Competitor_Click['Campaign'].apply(lambda x: x[len(x)-4:len(x)+1])=="[SM]"]
Saatva_Competitor_Click.reset_index(inplace=True)
del Saatva_Competitor_Click['index']
# Keep only campaign labeled as "[SM]" in Saatva Competitor account
# Broad and Exact ADL Non-Brand Search Partners are not included

In [31]:
set(Saatva_Click['Channel_Network'].unique().tolist()+Saatva_Brand_Click['Channel_Network'].unique().tolist()+Saatva_Competitor_Click['Channel_Network'].unique().tolist()+\
    TheSaatvaCompany_Click['Channel_Network'].unique().tolist()+TheSaatvaCompanyBranding_Click['Channel_Network'].unique().tolist())

{'Display|Display Network|Display Network',
 'Search|Search Network|Google search',
 'Search|Search Network|Search partners',
 'Shopping|Search Network|Google search',
 'Shopping|Search Network|Search partners',
 'Video|Display Network|Display Network',
 'Video|YouTube Search|YouTube Search',
 'Video|YouTube Videos|YouTube Videos'}

In [32]:
df_all_adwords=Saatva_Click.append(Saatva_Brand_Click).append(Saatva_Competitor_Click)
df_all_adwords=df_all_adwords.append(TheSaatvaCompany_Click).append(TheSaatvaCompanyBranding_Click)

In [33]:
df_s=df_all_adwords.groupby(['Channel_Network'])[['Impressions','Clicks','Cost']].sum().reset_index()
df_s[df_s['Channel_Network'].apply(lambda x: "Video" in x)]

,Channel_Network,Impressions,Clicks,Cost
5,Video|Display Network|Display Network,15333,42,75.592500
6,Video|YouTube Search|YouTube Search,523478,34,2080.090000
7,Video|YouTube Videos|YouTube Videos,29509959,133907,394547.707408


In [34]:
df_all_adwords.head(2)

,Account,Campaign,Day,Hour of day,Advertising Channel,Network,Network (with search partners),Impressions,Clicks,Cost,Channel_Network
0,Saatva,[ADL] [Display] [US] Remarketing - Static [CPA],2018-09-17,13,Display,Display Network,Display Network,2894,8,14.61,Display|Display Network|Display Network
1,Saatva,[ADL] [Non-Brand] [US] Mattresses - Eco (Broad...,2018-10-07,11,Search,Search Network,Google search,5,0,0.00,Search|Search Network|Google search


In [35]:
df_all_adwords[df_all_adwords['Channel_Network']=="Video|Display Network|Display Network"]['Cost'].sum()

75.592500000000001

In [36]:
df_all_adwords[df_all_adwords['Channel_Network']=="Video|YouTube Search|YouTube Search"]['Cost'].sum()

2080.0900000000001

In [37]:
df_all_adwords[df_all_adwords['Channel_Network']=="Video|YouTube Videos|YouTube Videos"]['Cost'].sum()

394547.70740800002

In [38]:
df_all_adwords[df_all_adwords['Channel_Network']=="Video|Display Network|Display Network"]['Day'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date()).min()

datetime.date(2018, 9, 1)

In [39]:
df_all_adwords[df_all_adwords['Channel_Network']=="Video|Display Network|Display Network"]['Day'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date()).max()

datetime.date(2018, 9, 19)

In [40]:
df_all_adwords[df_all_adwords['Channel_Network']=="Video|YouTube Search|YouTube Search"]['Day'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date()).min()

datetime.date(2018, 9, 1)

In [41]:
df_all_adwords[df_all_adwords['Channel_Network']=="Video|YouTube Search|YouTube Search"]['Day'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date()).max()

datetime.date(2018, 10, 11)

In [42]:
df_all_adwords[df_all_adwords['Channel_Network']=="Video|YouTube Videos|YouTube Videos"]['Day'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date()).min()

datetime.date(2018, 9, 1)

In [43]:
df_all_adwords[df_all_adwords['Channel_Network']=="Video|YouTube Videos|YouTube Videos"]['Day'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date()).max()

datetime.date(2018, 10, 11)

In [44]:
df_x=df_all_adwords[df_all_adwords['Channel_Network']=="Video|YouTube Videos|YouTube Videos"]
df_x['Day']=df_x['Day'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
df_x[df_x['Day']<datetime.date(2018,9,19)]['Cost'].sum()

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


183281.06436699998

In [45]:
df_x[df_x['Day']<datetime.date(2018,9,19)]

,Account,Campaign,Day,Hour of day,Advertising Channel,Network,Network (with search partners),Impressions,Clicks,Cost,Channel_Network
1458,Saatva,[ADL] [Video] [In-Stream] [US] Remarketing - L...,2018-09-18,17,Video,YouTube Videos,YouTube Videos,555,4,13.598348,Video|YouTube Videos|YouTube Videos
1516,Saatva,[ADL] [Video] [In-Stream] [US] Remarketing - L...,2018-09-15,19,Video,YouTube Videos,YouTube Videos,536,4,11.654840,Video|YouTube Videos|YouTube Videos
1544,Saatva,[ADL] [Video] [In-Stream] [US] Remarketing - L...,2018-09-16,23,Video,YouTube Videos,YouTube Videos,518,1,10.904270,Video|YouTube Videos|YouTube Videos
...,...,...,...,...,...,...,...,...,...,...,...
36551,The Saatva Company Branding,[ADL] [Video] [Action] Anthem Video Test HHI,2018-09-08,19,Video,YouTube Videos,YouTube Videos,2743,5,20.104041,Video|YouTube Videos|YouTube Videos
36554,The Saatva Company Branding,[ADL] [Video] [Action] Anthem Video Test HHI,2018-09-08,0,Video,YouTube Videos,YouTube Videos,1947,5,12.943972,Video|YouTube Videos|YouTube Videos
36598,The Saatva Company Branding,[ADL] [Video] [Action] Anthem Video Test HHI,2018-09-09,7,Video,YouTube Videos,YouTube Videos,974,6,7.553884,Video|YouTube Videos|YouTube Videos


In [46]:
'''
Saatva_Click=Saatva_Click[~((Saatva_Click['Campaign'].str.contains("Loom")) | (Saatva_Click['Campaign'].str.contains("Leaf")) | (Saatva_Click['Campaign'].str.contains("Zenhaven")))]
TheSaatvaCompany_Click=TheSaatvaCompany_Click[~((TheSaatvaCompany_Click['Campaign'].str.contains("Loom")) | (TheSaatvaCompany_Click['Campaign'].str.contains("Leaf")) | (TheSaatvaCompany_Click['Campaign'].str.contains("Zenhaven")))]
TheSaatvaCompanyBranding_Click=TheSaatvaCompanyBranding_Click[~((TheSaatvaCompanyBranding_Click['Campaign'].str.contains("Loom")) | (TheSaatvaCompanyBranding_Click['Campaign'].str.contains("Leaf")) | (TheSaatvaCompanyBranding_Click['Campaign'].str.contains("Zenhaven")))]
'''
# No need to exclude the compaigns because of the small pctg

Saatva_Click['Campaign_Type']=Saatva_Click['Advertising Channel'].apply(lambda x: x.replace("Search","SEM").replace("Shopping","PLA"))
Saatva_Brand_Click['Campaign_Type']=Saatva_Brand_Click['Advertising Channel'].apply(lambda x: x.replace("Search","SEM_Brand").replace("Shopping","PLA"))
Saatva_Competitor_Click['Campaign_Type']="SEM_Non_Brand" # Because it's all labeled as Non-Brand in Campaign and in Search Only Campaign Type
TheSaatvaCompany_Click['Campaign_Type']=TheSaatvaCompany_Click['Advertising Channel'].apply(lambda x: x.replace("Search","SEM").replace("Shopping","PLA"))
TheSaatvaCompanyBranding_Click['Campaign_Type']=TheSaatvaCompanyBranding_Click['Advertising Channel'].apply(lambda x: x.replace("Search","SEM").replace("Shopping","PLA"))


In [47]:
# Split Saatva
# 'Search Network with Display Select' Campaign Type all 0 Impr and Clicks
Saatva_Click_Search=Saatva_Click[Saatva_Click['Advertising Channel']=='Search']
Saatva_Click_Others=Saatva_Click[Saatva_Click['Advertising Channel']!='Search']

Saatva_Click_Search['Campaign_Type']=np.where((Saatva_Click_Search['Campaign'].str.contains("Branded")) | (Saatva_Click_Search['Campaign'].str.contains(" Brand ")) | (Saatva_Click_Search['Campaign'].str.contains("Co-Brand")) | (Saatva_Click_Search['Campaign'].str.contains("\[Brand\]")),"SEM_Brand",
                                                 np.where((Saatva_Click_Search['Campaign'].str.contains("Non-Brand")) | (Saatva_Click_Search['Campaign'].str.contains("Dynamic Search Ads")),"SEM_Non_Brand","NA"))
Saatva_Click_Search_1=Saatva_Click_Search[Saatva_Click_Search['Campaign_Type']!="NA"]
Saatva_Click_Search_2=Saatva_Click_Search[Saatva_Click_Search['Campaign_Type']=="NA"]
len(Saatva_Click_Search_2) # Make sure it's 0

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


0

In [48]:
# Split TheSaatvaCompany

TheSaatvaCompany_Click_Search=TheSaatvaCompany_Click[TheSaatvaCompany_Click['Advertising Channel']=='Search']
TheSaatvaCompany_Click_Others=TheSaatvaCompany_Click[TheSaatvaCompany_Click['Advertising Channel']!='Search']

TheSaatvaCompany_Click_Search['Campaign_Type']=np.where((TheSaatvaCompany_Click_Search['Campaign'].str.contains("Branded")) | (TheSaatvaCompany_Click_Search['Campaign'].str.contains(" Brand ")) | (TheSaatvaCompany_Click_Search['Campaign'].str.contains("Co-Brand")) | (TheSaatvaCompany_Click_Search['Campaign'].str.contains("\[Brand\]")),"SEM_Brand",
                                                 np.where((TheSaatvaCompany_Click_Search['Campaign'].str.contains("Non-Brand")) | (TheSaatvaCompany_Click_Search['Campaign'].str.contains("Dynamic Search Ads")),"SEM_Non_Brand","NA"))
TheSaatvaCompany_Click_Search_1=TheSaatvaCompany_Click_Search[TheSaatvaCompany_Click_Search['Campaign_Type']!="NA"]
TheSaatvaCompany_Click_Search_2=TheSaatvaCompany_Click_Search[TheSaatvaCompany_Click_Search['Campaign_Type']=="NA"]
len(TheSaatvaCompany_Click_Search_2) # Make sure it's 0

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


0

In [49]:
# All campaing is able to differentiate from the name about brand and non-brand, so no need to go to keywords level
'''
Saatva_keywords=pd.read_csv("/home/jian/Projects/Saatva/To_Lexie_and_Connor/AdWords/saatva_keywords.csv",dtype=str,skiprows=2)
Saatva_Brand_keywords=pd.read_csv("/home/jian/Projects/Saatva/To_Lexie_and_Connor/AdWords/saatva branding_keywords.csv",dtype=str)
Saatva_Competitor_keywords=pd.read_csv("/home/jian/Projects/Saatva/To_Lexie_and_Connor/AdWords/saatva competitors_keywords.csv",dtype=str,skiprows=2)

Saatva_keywords=Saatva_keywords[~((Saatva_keywords['Campaign'].str.contains("Loom")) | (Saatva_keywords['Campaign'].str.contains("Leaf")) | (Saatva_keywords['Campaign'].str.contains("Zenhaven")))]
Saatva_keywords['Campaign_Type']=np.where(Saatva_keywords['Search keyword'].str.contains("aatva"),"SEM_Brand","SEM_Non_Brand")
Saatva_keywords['Impressions']=Saatva_keywords['Impressions'].apply(lambda x: int(x.replace(",","")))
Saatva_keywords['Impressions']=Saatva_keywords['Impressions'].astype(int)
Saatva_keywords['Clicks']=Saatva_keywords['Clicks'].apply(lambda x: int(x.replace(",","")))
Saatva_keywords['Clicks']=Saatva_keywords['Clicks'].astype(int)
Saatva_keywords=Saatva_keywords.groupby(['Campaign','Campaign_Type'])['Impressions','Clicks'].sum()
Saatva_keywords.reset_index(inplace=True)
Saatva_keywords=Saatva_keywords.sort_values(['Campaign','Impressions'])
Saatva_keywords=Saatva_keywords.drop_duplicates('Campaign')
Saatva_Campaign_Type=Saatva_keywords[['Campaign','Campaign_Type']]

del Saatva_Click_Search_2['Campaign_Type']
Saatva_Click_Search_2=pd.merge(Saatva_Click_Search_2,Saatva_Campaign_Type,on="Campaign",how="left")

Saatva_Search_Click=Saatva_Click_Search_1.append(Saatva_Click_Search_2)
Saatva_Search_Click['Campaign_Type']=Saatva_Search_Click['Campaign_Type'].fillna("All_0_Values_and_No_Kws")
'''

'\nSaatva_keywords=pd.read_csv("/home/jian/Projects/Saatva/To_Lexie_and_Connor/AdWords/saatva_keywords.csv",dtype=str,skiprows=2)\nSaatva_Brand_keywords=pd.read_csv("/home/jian/Projects/Saatva/To_Lexie_and_Connor/AdWords/saatva branding_keywords.csv",dtype=str)\nSaatva_Competitor_keywords=pd.read_csv("/home/jian/Projects/Saatva/To_Lexie_and_Connor/AdWords/saatva competitors_keywords.csv",dtype=str,skiprows=2)\n\nSaatva_keywords=Saatva_keywords[~((Saatva_keywords[\'Campaign\'].str.contains("Loom")) | (Saatva_keywords[\'Campaign\'].str.contains("Leaf")) | (Saatva_keywords[\'Campaign\'].str.contains("Zenhaven")))]\nSaatva_keywords[\'Campaign_Type\']=np.where(Saatva_keywords[\'Search keyword\'].str.contains("aatva"),"SEM_Brand","SEM_Non_Brand")\nSaatva_keywords[\'Impressions\']=Saatva_keywords[\'Impressions\'].apply(lambda x: int(x.replace(",","")))\nSaatva_keywords[\'Impressions\']=Saatva_keywords[\'Impressions\'].astype(int)\nSaatva_keywords[\'Clicks\']=Saatva_keywords[\'Clicks\'].apply(

In [50]:
Saatva_Click_Search_2.head(2)

,Account,Campaign,Day,Hour of day,Advertising Channel,Network,Network (with search partners),Impressions,Clicks,Cost,Channel_Network,Campaign_Type


In [51]:
Saatva_Click=Saatva_Click_Search.append(Saatva_Click_Others)
TheSaatvaCompany_Click=TheSaatvaCompany_Click_Search.append(TheSaatvaCompany_Click_Others)


In [52]:
writer=pd.ExcelWriter(writer_folder+"/Saatva_AdWords_"+Start_Date_overall+"_to_"+End_Data_overall+week+"_cost.xlsx",engine='xlsxwriter')

In [53]:
Saatva_Agg=Saatva_Click.append(Saatva_Brand_Click).append(Saatva_Competitor_Click).append(TheSaatvaCompany_Click).append(TheSaatvaCompanyBranding_Click)
Saatva_Agg['Day']=Saatva_Agg['Day'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
Saatva_Agg.reset_index(inplace=True)
del Saatva_Agg['index']
Saatva_Agg['Hour of day']=Saatva_Agg['Hour of day'].astype(int)
Saatva_Agg_Category=Saatva_Agg[['Campaign','Account','Advertising Channel','Campaign_Type']].drop_duplicates()
Saatva_Agg_Category.to_excel(writer,"Campaign Category",index=False)

In [54]:
Saatva_Agg=Saatva_Agg.groupby(["Campaign_Type","Day","Hour of day"])['Impressions','Clicks', 'Cost'].apply(sum)
Saatva_Agg.reset_index(inplace=True)
Saatva_Agg['CTR']=Saatva_Agg['Clicks']/Saatva_Agg['Impressions']
Saatva_Agg.to_excel(writer,"reshaped by hour day",index=False)

Saatva_Agg_No_NA=Saatva_Agg[(Saatva_Agg['Campaign_Type']!="NA") & (Saatva_Agg['Campaign_Type']!="Search Network with Display Select")]

Saatva_Agg_No_NA.to_excel(writer,"clean data by hour day",index=False)

In [55]:
writer.save()

In [56]:
AdWords_output=Saatva_Agg.copy()
AdWords_output['Date']=AdWords_output['Day'].apply(lambda x: str(x)[0:4]+str(x)[5:7]+str(x)[8:10])
AdWords_output=AdWords_output[['Campaign_Type','Date','Hour of day','Impressions','Clicks','Cost']]
AdWords_output=AdWords_output.rename(columns={"Hour of day":"Hour","Impressions":"IMP"})
AdWords_output=AdWords_output.sort_values(["Campaign_Type","Date","Hour"],ascending=[True,True,True])
AdWords_output.to_csv(writer_folder+"/Saatva_AdWords_"+Start_Date_overall+"_to_"+End_Data_overall+week+"_cost.csv",index=False)